## Initiate the Toolbox
- the parameters set here will be used to collect command-specific data when the `Fetcher` executes the TET pattern.

In [1]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbols=["AAPL", "AMD", "GOOGL", "PCT", "NVDA"],
    interval="1d",
    start_date="2020-01-01",
    end_date="2024-01-01",
)
toolbox

Toolbox(symbols=['AAPL', 'AMD', 'GOOGL', 'PCT', 'NVDA'], interval=1d, start_date=2020-01-01, end_date=2024-01-01, provider=yfinance)

## Run Mandelbrot Channel Calculation

In [3]:
result = toolbox.technical.mandelbrot_channel(historical=True, window="1m", _boundary_group_down=False)
result
result.to_polars()

2024-07-08 16:32:11,108 - MandelbrotChannelFetcher - INFO - START: fetch_data (sync)


2024-07-08 16:32:17,965 - MandelbrotChannelFetcher - INFO - END: fetch_data (sync) - Total time: 6.8576s


date,symbol,bottom_price,close_price,top_price
date,str,f64,f64,f64
2020-02-03,"""AAPL""",74.9288,74.979103,80.7142
2020-02-04,"""AAPL""",74.2665,77.454445,82.2336
2020-02-05,"""AAPL""",74.6945,78.086052,82.3749
2020-02-06,"""AAPL""",75.0301,78.999413,82.5859
2020-02-07,"""AAPL""",75.4156,77.925583,82.594
…,…,…,…,…
2023-12-22,"""PCT""",3.2567,3.84,4.3722
2023-12-26,"""PCT""",3.3069,3.96,4.46
2023-12-27,"""PCT""",3.3973,4.11,4.5761
